## Import libraries

In [ ]:
# harp-python for data interface and pandas for simple plotting
import harp
import pandas as pd 

## Load device reader

In [ ]:
device = harp.create_reader("Data/Hobgoblin.harp")

## Load register data

In [ ]:
# The data for each register can be loaded into a pandas dataframe by simply calling read() on the register name
# For a full list of registers, refer to the API documentation at https://harp-tech.org/api/Harp.Hobgoblin.html
digital_output_set = device.DigitalOutputSet.read()
digital_output_clear = device.DigitalOutputClear.read()
digital_input_state = device.DigitalInputState.read()

## Inspect dataframes

In [ ]:
print("LED onset time")
print(digital_output_set.head())
print("\nLED offset time")
print(digital_output_clear.head())
print("\nButton press time")
print(digital_input_state.head())

## Preprocess dataframes

In [ ]:
# Discard unused channels
digital_output_set = digital_output_set["GP15"]
digital_output_clear = digital_output_clear["GP15"]
digital_input_state = digital_input_state["GP2"]

# Keep button presses
digital_input_state = digital_input_state[digital_input_state == True]

## Calculate hit response time

In [ ]:
# Keep only 1st button press that occurs within response_window/timeout
valid_response_times = []
for trial_number in range(len(digital_output_set.index)):
    led_on = digital_output_set.index[trial_number]
    response_window = digital_output_clear.index[trial_number] - digital_output_set.index[trial_number]
    for button_press in digital_input_state.index:
        response_time = button_press - led_on
        if 0 < response_time < response_window:
            valid_response_times.append(response_time)
            break

## Calculate hit/miss percentage

In [ ]:
num_valid_responses = len(valid_response_times)
num_total_trials = len(digital_output_set.index)
hit_percentage = num_valid_responses / num_total_trials * 100
print(f"There were {num_valid_responses} valid responses out of {num_total_trials} trials, giving a hit rate of {hit_percentage}%")

## Plot hit response times

In [ ]:
pd.Series(valid_response_times).plot(kind="hist")